#Instalar Dependencias

In [ ]:
!pip install -r requeriments.txt

## Crear BD y Cargar Usuarios de Ejemplo

In [54]:
import sqlite3
from faker import Faker
import json
from datetime import datetime

# Initialize Faker
fake = Faker()

# Connect to the database
conn = sqlite3.connect('tables.db')
cursor = conn.cursor()

# Create Tables
def create_tables():
    """Crea las tablas necesarias si no existen"""

    # Tabla de usuarios
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS usuarios (
        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        telefono TEXT,
        redes_sociales TEXT,
        fecha_nacimiento TEXT,
        genero TEXT,
        ocupacion TEXT,
        deportes TEXT,
        presupuesto_maximo REAL,
        habitos_limpieza INTEGER CHECK(habitos_limpieza BETWEEN 1 AND 5),
        horario_trabajo TEXT,
        tiene_mascota INTEGER,
        acepta_mascota INTEGER,
        es_fumador INTEGER,
        acepta_fumador INTEGER,
        intereses TEXT,
        preferencias_roommate TEXT,
        fecha_registro TEXT,
        ultima_actualizacion TEXT,
        activo INTEGER DEFAULT 1
    )
    ''')

    # Tabla de similitudes
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS similitudes (
        user_id_1 INTEGER,
        user_id_2 INTEGER,
        score_similitud REAL,
        fecha_calculo TEXT,
        PRIMARY KEY (user_id_1, user_id_2),
        FOREIGN KEY (user_id_1) REFERENCES usuarios(user_id),
        FOREIGN KEY (user_id_2) REFERENCES usuarios(user_id)
    )
    ''')
    conn.commit()

# Function to generate a single user
def generate_user():
    user = {
        'nombre': fake.name(),
        'email': fake.email(),
        'telefono': fake.phone_number(),
        'redes_sociales': fake.user_name(),
        'fecha_nacimiento': fake.date_of_birth(minimum_age=18, maximum_age=40).isoformat(),
        'genero': fake.random_element(elements=('M', 'F')),
        'ocupacion': fake.job(),
        'deportes': fake.random_element(elements=('Yoga', 'Running', 'Natación', 'Fútbol', 'Baloncesto', '')),
        'presupuesto_maximo': fake.pyfloat(min_value=300, max_value=1000, right_digits=2),
        'habitos_limpieza': fake.random_int(min=1, max=5),
        'horario_trabajo': fake.random_element(elements=('9:00-17:00', '10:00-18:00', '8:00-16:00', '')),
        'tiene_mascota': fake.boolean(),
        'acepta_mascota': fake.boolean(),
        'es_fumador': fake.boolean(),
        'acepta_fumador': fake.boolean(),
        'intereses': json.dumps(fake.words(nb=3)),
        'preferencias_roommate': json.dumps({'edad_min': fake.random_int(min=18, max=30),
                                            'edad_max': fake.random_int(min=30, max=50)}),
        'fecha_registro': datetime.now().isoformat(),
        'ultima_actualizacion': datetime.now().isoformat(),
        'activo': 1
    }
    return user


create_tables()
# Insert 100 users
for _ in range(100):
    user = generate_user()
    try:
        cursor.execute("INSERT INTO usuarios (nombre, email, telefono, redes_sociales, fecha_nacimiento, genero, ocupacion, deportes, presupuesto_maximo, habitos_limpieza, horario_trabajo, tiene_mascota, acepta_mascota, es_fumador, acepta_fumador, intereses, preferencias_roommate, fecha_registro, ultima_actualizacion, activo) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", list(user.values()))
    except sqlite3.IntegrityError:  # Handle potential unique constraint violations
        pass  # Skip if email is already in the database
conn.commit()

# deshabilita 3 usuarios
user_ids = (50, 51, 52)
cursor.execute("UPDATE usuarios SET activo = 0 WHERE user_id IN (?, ?, ?)", user_ids)


# Commit changes and close connection
conn.commit()
conn.close()

print("100 example users inserted into the database.")


100 example users inserted into the database.


## Crear Tabla Recomendaciones, y calcular todos las relaciones

In [55]:
import sqlite3
import re

def recomendacion_score(usr1, usr2):
    score=0.0
    # Redes sociales [parametro 4]
    if usr1[4] and usr2[4]:  # Ambos tienen redes sociales definidas
        score += 0.6
    elif not usr1[4] and not usr2[4]:  # Ninguno tiene redes sociales definidas
        score += 0.5


    # Edad [parametro 5]
    try:
        # Convierte las fechas de nacimiento a objetos datetime
        fecha_nacimiento1 = datetime.fromisoformat(usr1[5])
        fecha_nacimiento2 = datetime.fromisoformat(usr2[5])

        # Calcula la diferencia de edad en años decimales
        diferencia_edad=(fecha_nacimiento1-fecha_nacimiento2).days / 365.25

        # Aplica la función exponencial para la similitud de edad
        score += 2 * np.exp(-abs(diferencia_edad) / 3)

    except (TypeError, ValueError, IndexError):
        pass  # Ignora si la fecha de nacimiento no es válida o no está presente

    # Género [parametro 6]
    if usr1[6] and usr2[6] and usr1[6] == usr2[6]:
        score += 1.0

    # Ocupación [parametro 7]
    if usr1[7] and usr2[7] and comparador_cadena_palabras(usr1[7], usr2[7]):
        score += 2.0

    # Deportes [parametro 8]
    if usr1[8] and usr2[8] and comparador_cadena_palabras(usr1[8], usr2[8]):
        score += 2.0

    # Presupuesto [parametro 9]
    if abs(usr1[9] - usr2[9]) < 100:
        score += 1.0

    # Hábitos de limpieza [parametro 10]
    if usr1[10] and usr2[10] and usr1[10] == usr2[10]:  #si comparten tipo limpieza
        score += 1.0

    if usr1[10] and usr2[10] and abs(usr1[10]-usr2[10])==1:  # si son similares
        score += 0.8

    if usr1[10] and usr2[10] and abs(usr1[10]-usr2[10])==2:  # si difieren por dos
        score += 0.4

    # Horario de trabajo [parametro 11]
    if usr1[11] and usr2[11] and usr1[11] == usr2[11]:
        score += 1

    # Mascotas [parametro 12:tiene_mascota] [parametro 13:acepta_mascota]
    if usr1[12] and not usr2[13]:  # usr1 tiene mascota y usr2 no la acepta
        score -= 2.0
    if usr2[12] and not usr1[13]:  # usr2 tiene mascota y usr1 no la acepta
        score -= 2.0
    if usr1[13] and usr2[12]:  # usr1 acepta mascota y usr2 tiene una
        score += 0.5
    if usr2[13] and usr1[12]:  # usr2 acepta mascota y usr1 tiene una
        score += 0.5
    if usr1[13] == usr2[13]:  # Comparten gusto por mascota
        score += 1.0

    # Fumador [parametro 12:es_fumador] [parametro 13:acepta_fumador]
    if usr1[14] and not usr2[15]:  # usr1 fuma y usr2 no lo acepta
        score -= 2.0
    if usr2[14] and not usr1[15]:  # usr2 fuma y usr1 no lo acepta
        score -= 2.0
    if usr1[15] and usr2[14]:  # usr1 acepta fumador y usr2 fuma
        score += 0.5
    if usr2[15] and usr1[14]:  # usr2 acepta fumador y usr1 fuma
        score += 0.5
    if usr1[15] == usr2[15]:  # Comparten gusto sobre fumar
        score += 1.0

    # Intereses [parametro 16]
    try:
        intereses1 = json.loads(usr1[16]) if usr1[16] else [] # Si usr1[16] es None o una cadena vacía, inicializar intereses1 como una lista vacía.
        intereses2 = json.loads(usr2[16]) if usr2[16] else [] # Lo mismo para intereses2

        if intereses1 and intereses2 and comparador_cadena_palabras(intereses1, intereses2):
           score += 2.0
    except (json.JSONDecodeError, TypeError):
        pass  # Ignora si los intereses no son un JSON válido o si no están presentes

    # Preferencias de roommate [parametro 17]
    try:
        preferencias1 = json.loads(usr1[17]) if usr1[17] else {}
        preferencias2 = json.loads(usr2[17]) if usr2[17] else {}

        if preferencias1 and preferencias2 and comparador_cadena_palabras(preferencias1, preferencias2):
           score += 2.0
    except (json.JSONDecodeError, TypeError):
        pass  # Ignora si las preferencias no son un JSON válido o si no están presentes

    return score

    #Compara dos cadenas de texto (o listas) y busca palabras en común.
def comparador_cadena_palabras(str1, str2):
    def _extract_words(data):
        """Extrae palabras de una cadena, lista o diccionario."""
        if isinstance(data, str):
            return re.findall(r'\b\w+\b', data.lower())
        elif isinstance(data, list):
            return re.findall(r'\b\w+\b', ", ".join(data).lower())
        elif isinstance(data, dict):
            words = []
            for value in data.values():
                if isinstance(value, str):
                    words.extend(re.findall(r'\b\w+\b', value.lower()))
                elif isinstance(value, list):
                    words.extend(re.findall(r'\b\w+\b', ", ".join(value).lower()))
            return words
        else:
            return []

    palabras1 = _extract_words(str1)
    palabras2 = _extract_words(str2)

    # Buscar palabras en común
    palabras_comunes = set(palabras1) & set(palabras2)

    # Si hay al menos una palabra en común, retornar True
    return len(palabras_comunes) > 0

# Conexión a la base de datos
conn = sqlite3.connect('tables.db')
cursor = conn.cursor()

# Obtener los IDs de usuarios activos
cursor.execute("SELECT * FROM usuarios WHERE activo = 1")
usuarios_activos = cursor.fetchall()

# Crear una lista para almacenar las similaridades
similaridades = []

# Calcular la similitud entre todos los pares de usuarios activos
for i in range(len(usuarios_activos)):
    for j in range(i + 1, len(usuarios_activos)):
        usr1 = usuarios_activos[i]
        usr2 = usuarios_activos[j]
        score = recomendacion_score(usr1, usr2)
        similaridades.append((usr1[0], usr2[0], score))

# Insertar las similaridades en la tabla 'similaridades'
for usr1_id, usr2_id, score in similaridades:
    try:
        cursor.execute("INSERT INTO similitudes (user_id_1, user_id_2, score_similitud, fecha_calculo) VALUES (?, ?, ?, datetime('now'))", (usr1_id, usr2_id, score))
    except sqlite3.IntegrityError:
        pass

# Guardar los cambios y cerrar la conexión
conn.commit()
conn.close()


##Ver tablas

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("tables.db")
query = "SELECT * FROM usuarios LIMIT 5"
print(pd.read_sql_query(query, conn).head(10))
query = "SELECT * FROM similitudes LIMIT 5"
print(pd.read_sql_query(query, conn).head(10))
conn.close()

## Consulta para **ver los 5 usuarios** mas compatibles con el usuario 5

In [62]:
import sqlite3
import pandas as pd

# Conectar a la base de datos
conn = sqlite3.connect('tables.db')

# Consulta sobre el usuario 5
query = "SELECT * FROM usuarios WHERE (user_id=5)"
print(pd.read_sql_query(query, conn))
print(""

"")

# Consulta SQL
query = """
SELECT u.*
FROM usuarios u
JOIN similitudes s ON u.user_id = s.user_id_1 OR u.user_id = s.user_id_2
WHERE (s.user_id_1 = 5 OR s.user_id_2 = 5) AND u.user_id != 5
ORDER BY s.score_similitud DESC
LIMIT 5
"""

df = pd.read_sql_query(query, conn)
print(df)

# Cerrar la conexión
conn.close()

   user_id      nombre                  email              telefono  \
0        5  Sue Arnold  abigail15@example.net  +1-398-616-0567x2111   

  redes_sociales fecha_nacimiento genero            ocupacion deportes  \
0       lskinner       1988-03-06      M  Scientist, forensic            

   presupuesto_maximo  ...  horario_trabajo tiene_mascota  acepta_mascota  \
0              689.42  ...       8:00-16:00             0               0   

   es_fumador  acepta_fumador                   intereses  \
0           0               1  ["people", "dark", "high"]   

              preferencias_roommate              fecha_registro  \
0  {"edad_min": 28, "edad_max": 38}  2025-03-11T19:37:34.513735   

         ultima_actualizacion activo  
0  2025-03-11T19:37:34.513739      1  

[1 rows x 21 columns]

   user_id            nombre                         email  \
0       60     Peter Leblanc  kimberlyhamilton@example.net   
1       84       Larry Jones      timothydavis@example.org   
2      

## Programa Principal

In [22]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import sqlite3
from sklearn.metrics.pairwise import cosine_similarity

class RoommateRecommendationSystem:
    def __init__(self, db_path=':memory:'):
        """Inicializa el sistema de recomendaciones con una conexión a la base de datos"""
        self.conn = sqlite3.connect(db_path)
        self.create_tables()

    def create_tables(self):
        """Crea las tablas necesarias si no existen"""
        cursor = self.conn.cursor()

        # Tabla de usuarios
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS usuarios (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT,
            nombre TEXT NOT NULL,
            email TEXT UNIQUE NOT NULL,
            telefono TEXT,
            redes_sociales TEXT,
            fecha_nacimiento TEXT,
            genero TEXT,
            ocupacion TEXT,
            deportes TEXT,
            presupuesto_maximo REAL,
            habitos_limpieza INTEGER CHECK(habitos_limpieza BETWEEN 1 AND 5),
            horario_trabajo TEXT,
            tiene_mascota INTEGER,
            acepta_mascota INTEGER,
            es_fumador INTEGER,
            acepta_fumador INTEGER,
            intereses TEXT,
            preferencias_roommate TEXT,
            fecha_registro TEXT,
            ultima_actualizacion TEXT,
            activo INTEGER DEFAULT 1
        )
        ''')

        # Tabla de similitudes
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS similitudes (
            user_id_1 INTEGER,
            user_id_2 INTEGER,
            score_similitud REAL,
            fecha_calculo TEXT,
            PRIMARY KEY (user_id_1, user_id_2),
            FOREIGN KEY (user_id_1) REFERENCES usuarios(user_id),
            FOREIGN KEY (user_id_2) REFERENCES usuarios(user_id)
        )
        ''')

        self.conn.commit()

    def create(self, params):
        """
        Crea un nuevo usuario y calcula su similitud con todos los usuarios existentes

        Args:
            params (dict): Diccionario con los datos del usuario

        Returns:
            int: ID del usuario creado
        """
        # Convertir listas y diccionarios a JSON para almacenamiento
        if 'intereses' in params and isinstance(params['intereses'], (dict, list)):
            params['intereses'] = json.dumps(params['intereses'])
        if 'preferencias_roommate' in params and isinstance(params['preferencias_roommate'], (dict, list)):
            params['preferencias_roommate'] = json.dumps(params['preferencias_roommate'])

        # Establecer fechas
        now = datetime.now().isoformat()
        params['fecha_registro'] = now
        params['ultima_actualizacion'] = now

        # Convertir booleanos a enteros para SQLite
        boolean_fields = ['tiene_mascota', 'acepta_mascota', 'es_fumador', 'acepta_fumador']
        for field in boolean_fields:
            if field in params:
                params[field] = 1 if params[field] else 0

        # Preparar la consulta SQL
        fields = ', '.join(params.keys())
        placeholders = ', '.join(['?'] * len(params))

        cursor = self.conn.cursor()
        query = f"INSERT INTO usuarios ({fields}) VALUES ({placeholders})"

        try:
            cursor.execute(query, list(params.values()))
            self.conn.commit()
            user_id = cursor.lastrowid

            # Calcular similitudes con usuarios existentes
            self._calculate_similarities(user_id)

            return user_id
        except Exception as e:
            self.conn.rollback()
            raise e

    def update(self, user_id, params):
        """
        Actualiza un usuario existente y recalcula sus similitudes

        Args:
            user_id (int): ID del usuario a actualizar
            params (dict): Diccionario con los datos a actualizar

        Returns:
            bool: True si la actualización fue exitosa
        """
        # Convertir listas y diccionarios a JSON para almacenamiento
        if 'intereses' in params and isinstance(params['intereses'], (dict, list)):
            params['intereses'] = json.dumps(params['intereses'])
        if 'preferencias_roommate' in params and isinstance(params['preferencias_roommate'], (dict, list)):
            params['preferencias_roommate'] = json.dumps(params['preferencias_roommate'])

        # Actualizar fecha de modificación
        params['ultima_actualizacion'] = datetime.now().isoformat()

        # Convertir booleanos a enteros para SQLite
        boolean_fields = ['tiene_mascota', 'acepta_mascota', 'es_fumador', 'acepta_fumador']
        for field in boolean_fields:
            if field in params:
                params[field] = 1 if params[field] else 0

        # Preparar la consulta SQL
        set_clause = ', '.join([f"{key} = ?" for key in params.keys()])

        cursor = self.conn.cursor()
        query = f"UPDATE usuarios SET {set_clause} WHERE user_id = ?"

        try:
            cursor.execute(query, list(params.values()) + [user_id])
            self.conn.commit()

            # Si se actualizó algún registro, recalcular similitudes
            if cursor.rowcount > 0:
                self._calculate_similarities(user_id)
                return True
            return False
        except Exception as e:
            self.conn.rollback()
            raise e

    def delete(self, user_id):
        """
        Marca un usuario como eliminado y elimina sus similitudes

        Args:
            user_id (int): ID del usuario a eliminar

        Returns:
            bool: True si la eliminación fue exitosa
        """
        cursor = self.conn.cursor()

        try:
            # Marcar usuario como inactivo (eliminación lógica)
            cursor.execute("UPDATE usuarios SET activo = 0 WHERE user_id = ?", (user_id,))

            # Eliminar similitudes asociadas
            cursor.execute("DELETE FROM similitudes WHERE user_id_1 = ? OR user_id_2 = ?",
                           (user_id, user_id))

            self.conn.commit()
            return cursor.rowcount > 0
        except Exception as e:
            self.conn.rollback()
            raise e

    def get_similaridad(self, user_id, limit=10):
        """
        Obtiene los usuarios más similares a un usuario dado

        Args:
            user_id (int): ID del usuario de referencia
            limit (int): Número máximo de resultados a retornar

        Returns:
            list: Lista de diccionarios con información de usuarios similares
        """
        cursor = self.conn.cursor()

        query = """
        SELECT u.user_id, u.nombre, u.email, s.score_similitud
        FROM similitudes s
        JOIN usuarios u ON (s.user_id_2 = u.user_id AND s.user_id_1 = ?)
                        OR (s.user_id_1 = u.user_id AND s.user_id_2 = ?)
        WHERE u.user_id != ? AND u.activo = 1
        ORDER BY s.score_similitud DESC
        LIMIT ?
        """

        cursor.execute(query, (user_id, user_id, user_id, limit))

        results = []
        for row in cursor.fetchall():
            results.append({
                'user_id': row[0],
                'nombre': row[1],
                'email': row[2],
                'score_similitud': row[3]
            })

        return results

    def _calculate_similarities(self, user_id):
        """
        Calcula la similitud entre un usuario y todos los demás usuarios activos

        Args:
            user_id (int): ID del usuario para el que se calculan similitudes
        """
        cursor = self.conn.cursor()

        # Obtener todos los usuarios activos
        cursor.execute("SELECT user_id FROM usuarios WHERE activo = 1")
        all_users = [row[0] for row in cursor.fetchall()]

        # Eliminar similitudes existentes
        cursor.execute("DELETE FROM similitudes WHERE user_id_1 = ? OR user_id_2 = ?",
                       (user_id, user_id))

        # Si solo hay un usuario, no hay similitudes que calcular
        if len(all_users) <= 1:
            self.conn.commit()
            return

        # Obtener características de todos los usuarios para vectorización
        user_data = self._get_user_features()

        # Calcular similitudes usando coseno
        similarities = self._compute_similarity_matrix(user_data)

        # Fecha actual para el registro
        now = datetime.now().isoformat()

        # Insertar nuevas similitudes
        new_similarities = []
        for other_id in all_users:
            if other_id != user_id:
                # Ordenar IDs para mantener consistencia en la base de datos
                user1, user2 = min(user_id, other_id), max(user_id, other_id)

                # Encontrar índices en la matriz de similitud
                idx1 = user_data.index[user_data['user_id'] == user1].tolist()[0]
                idx2 = user_data.index[user_data['user_id'] == user2].tolist()[0]

                score = similarities[idx1, idx2]

                new_similarities.append((user1, user2, score, now))

        cursor.executemany(
            "INSERT INTO similitudes (user_id_1, user_id_2, score_similitud, fecha_calculo) VALUES (?, ?, ?, ?)",
            new_similarities
        )

        self.conn.commit()

    def _get_user_features(self):
        """
        Obtiene las características de todos los usuarios para calcular similitudes

        Returns:
            DataFrame: DataFrame con las características de los usuarios
        """
        query = """
        SELECT
            user_id, presupuesto_maximo, habitos_limpieza,
            tiene_mascota, acepta_mascota, es_fumador, acepta_fumador,
            intereses, preferencias_roommate, genero, ocupacion, deportes
        FROM
            usuarios
        WHERE
            activo = 1
        """

        df = pd.read_sql_query(query, self.conn)

        # Procesar características categóricas y JSON
        # Convertir campos JSON a diccionarios Python
        if 'intereses' in df.columns:
            df['intereses'] = df['intereses'].apply(lambda x: json.loads(x) if x else {})

        if 'preferencias_roommate' in df.columns:
            df['preferencias_roommate'] = df['preferencias_roommate'].apply(lambda x: json.loads(x) if x else {})

        # Aquí se podrían realizar más transformaciones para preparar los datos
        # como one-hot encoding de variables categóricas, normalización, etc.

        return df

    def _compute_similarity_matrix(self, user_data):
        """
        Calcula la matriz de similitud entre usuarios

        Args:
            user_data (DataFrame): DataFrame con características de usuarios

        Returns:
            ndarray: Matriz de similitud
        """
        # Copia del DataFrame para procesamiento
        df = user_data.copy()

        # Extraer user_id para referencia
        user_ids = df['user_id'].values

        # Eliminar columnas que no se usarán para cálculos de similitud
        df = df.drop(['user_id', 'intereses', 'preferencias_roommate'], axis=1, errors='ignore')

        # Convertir campos categóricos usando one-hot encoding
        categorical_cols = ['genero', 'ocupacion', 'deportes']
        for col in categorical_cols:
            if col in df.columns:
                dummies = pd.get_dummies(df[col], prefix=col)
                df = pd.concat([df.drop(col, axis=1), dummies], axis=1)

        # Manejo de valores nulos
        df = df.fillna(0)

        # Implementación simple usando similitud de coseno
        # Para un sistema real, se recomendaría un enfoque más sofisticado
        similarity_matrix = cosine_similarity(df)

        return similarity_matrix

    def get_user_by_id(self, user_id):
        """
        Obtiene información de un usuario por su ID

        Args:
            user_id (int): ID del usuario

        Returns:
            dict: Información del usuario o None si no existe
        """
        cursor = self.conn.cursor()
        cursor.execute("SELECT * FROM usuarios WHERE user_id = ? AND activo = 1", (user_id,))

        columns = [column[0] for column in cursor.description]
        user = cursor.fetchone()

        if not user:
            return None

        user_dict = dict(zip(columns, user))

        # Convertir campos JSON a Python
        if 'intereses' in user_dict and user_dict['intereses']:
            user_dict['intereses'] = json.loads(user_dict['intereses'])

        if 'preferencias_roommate' in user_dict and user_dict['preferencias_roommate']:
            user_dict['preferencias_roommate'] = json.loads(user_dict['preferencias_roommate'])

        # Convertir campos booleanos de SQLite (0/1) a Python (False/True)
        boolean_fields = ['tiene_mascota', 'acepta_mascota', 'es_fumador', 'acepta_fumador', 'activo']
        for field in boolean_fields:
            if field in user_dict:
                user_dict[field] = bool(user_dict[field])

        return user_dict

    def close(self):
        """Cierra la conexión a la base de datos"""
        if self.conn:
            self.conn.close()


# Ejemplo de uso
if __name__ == "__main__":
    # Inicializar el sistema
    system = RoommateRecommendationSystem("roommates.db")

    # Crear un usuario de ejemplo
    user1 = {
        'nombre': 'Ana García',
        'email': 'ana.garcia@example.com',
        'telefono': '555-1234',
        'redes_sociales': '@anagarcia',
        'fecha_nacimiento': '1995-05-15',
        'genero': 'F',
        'ocupacion': 'Estudiante',
        'deportes': 'Yoga, Running',
        'presupuesto_maximo': 500.0,
        'habitos_limpieza': 4,
        'horario_trabajo': '9:00-17:00',
        'tiene_mascota': False,
        'acepta_mascota': True,
        'es_fumador': False,
        'acepta_fumador': False,
        'intereses': ['música', 'cine', 'lectura'],
        'preferencias_roommate': {'edad_min': 20, 'edad_max': 35, 'mismo_genero': True}
    }

    user2 = {
        'nombre': 'Carlos Pérez',
        'email': 'carlos.perez@example.com',
        'telefono': '555-5678',
        'genero': 'M',
        'ocupacion': 'Profesional',
        'presupuesto_maximo': 600.0,
        'habitos_limpieza': 3,
        'tiene_mascota': False,
        'acepta_mascota': False,
        'es_fumador': False,
        'acepta_fumador': False,
        'intereses': ['deporte', 'cine', 'viajes'],
        'preferencias_roommate': {'edad_min': 25, 'edad_max': 40}
    }

    # Crear usuarios
    user1_id = system.create(user1)
    user2_id = system.create(user2)

    print(f"Usuario 1 creado con ID: {user1_id}")
    print(f"Usuario 2 creado con ID: {user2_id}")

    # Actualizar un usuario
    system.update(user1_id, {'presupuesto_maximo': 550.0, 'habitos_limpieza': 5})

    # Obtener similitudes
    similitudes = system.get_similaridad(user1_id)
    print("\nUsuarios similares a Ana García:")
    for similar in similitudes:
        print(f"{similar['nombre']} ({similar['email']}): {similar['score_similitud']:.2f}")

    # Cerrar conexión
    system.close()


Usuario 1 creado con ID: 1
Usuario 2 creado con ID: 2

Usuarios similares a Ana García:
Carlos Pérez (carlos.perez@example.com): 1.00
